In [ ]:

import sys
import re
import codecs
import string
import os
import numpy as np
from sklearn import svm
from sklearn.linear_model import *
from sklearn.naive_bayes import *
from sklearn import metrics
from sklearn.ensemble import *
from sklearn import cross_validation
import gzip
#from happyfuntokenizing import *

mycompile = lambda pat:  re.compile(pat,  re.UNICODE)

PRONOUN_PATH = '/home/sreenu/Desktop/rudra/koustav_phdthesis_2018-master/Lexical_resources/devanagari/devanagari_personal_pronoun.txt'
WHWORD_PATH = '/home/sreenu/Desktop/rudra/koustav_phdthesis_2018-master/Lexical_resources/devanagari/devanagari_wh_words.txt'
SLANG_PATH = '/home/sreenu/Desktop/rudra/koustav_phdthesis_2018-master/Lexical_resources/devanagari/devanagari_slang.txt'
INTENSIFIER_PATH = '/home/sreenu/Desktop/rudra/koustav_phdthesis_2018-master/Lexical_resources/devanagari/devanagari_intensifier.txt'
SUBJECTIVE_PATH = '/home/sreenu/Desktop/rudra/koustav_phdthesis_2018-master/Lexical_resources/devanagari/devanagari_subjective.txt'
EVENT_PATH = '/home/sreenu/Desktop/rudra/koustav_phdthesis_2018-master/Lexical_resources/devanagari/devanagari_domain_list.txt'
RELIGION_PATH = '/home/sreenu/Desktop/rudra/koustav_phdthesis_2018-master/Lexical_resources/devanagari/devanagari_religion.txt'
MODAL_VERB_PATH = '/home/sreenu/Desktop/rudra/koustav_phdthesis_2018-master/Lexical_resources/devanagari/devanagari_modal_verb.txt'

PRONOUN = {}
INTENSIFIER = {}
WHWORD = {}
SLANG = {}
EVENT = {}
MODAL = {}
RELIGION = {}
SUBJECTIVE = {}

emoticon_string = r"""
    (?:
      [<>]?
      [:;=8]                     # eyes
      [\-o\*\']?                 # optional nose
      [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth      
      |
      [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth
      [\-o\*\']?                 # optional nose
      [:;=8]                     # eyes
      [<>]?
    )"""

#TAGGER_PATH = '/home/krudra/twitter_code/term_project/ark-tweet-nlp-0.3.2/'

def getNumberofHashTags(s):
    return len(re.findall(r"""(?:\#+[\w_]+[\w\'_\-]*[\w_]+)""", s))

def getNumberOfCaps(s):
    return len(re.findall("[A-Z]+", s))

def getNumberOfPunctuations(s):
    punctuations= string.punctuation
    punctuations = re.sub("[#@]", "", punctuations)
    #print punctuations
    return len(re.findall("['%s']"%punctuations, s))

#def getNumberOfSmileys(s):
#    return len(re.findall(emoticon_string, s))

def getNumberOfPositiveSmileys(s):
    NormalEyes = r'[:=]'
    #Wink = r'[;]'
    NoseArea = r'(|o|O|-)'   ## rather tight precision, \S might be reasonable...
    HappyMouths = r'[D\)\]]'
    #SadMouths = r'[\(\[]'
    #Tongue = r'[pP]'
    #OtherMouths = r'[doO/\\]'  # remove forward slash if http://'s aren't cleaned
    Happy_RE =  mycompile( '(\^_\^|' + NormalEyes + NoseArea + HappyMouths + ')')
    #Sad_RE = mycompile(NormalEyes + NoseArea + SadMouths)
    
#     Wink_RE = mycompile(Wink + NoseArea + HappyMouths)
#     Tongue_RE = mycompile(NormalEyes + NoseArea + Tongue)
#     Other_RE = mycompile( '('+NormalEyes+'|'+Wink+')'  + NoseArea + OtherMouths )
#     
#     Emoticon = (
#         "("+NormalEyes+"|"+Wink+")" +
#         NoseArea + 
#         "("+Tongue+"|"+OtherMouths+"|"+SadMouths+"|"+HappyMouths+")"
#     )
#     Emoticon_RE = mycompile(Emoticon)
# 
#     Emoticon_RE = "|".join([Happy_RE,Sad_RE,Wink_RE,Tongue_RE,Other_RE])
#     Emoticon_RE = mycompile(Emoticon_RE)
    return len(re.findall(Happy_RE, s))    

def getNumberOfNegativeSmileys(s):
    NormalEyes = r'[:=]'
    NoseArea = r'(|o|O|-)'   ## rather tight precision, \S might be reasonable...
    SadMouths = r'[\(\[]'
    Sad_RE = mycompile(NormalEyes + NoseArea + SadMouths)
    return len(re.findall(Sad_RE, s))    

#print getNumberofHashTags("#unni_k #win AB")
#print getNumberOfCaps("#unni_k #win AB CC")
#print getNumberOfPunctuations("!!?")
#print getNumberOfSmileys(":):P")
#print getNumberOfElongatedWords("sooo muchhhhh")
#print getNumberOfPositiveSmileys(":) :( :P :-) :D :P")
#print getNumberOfNegativeSmileys(":( :) :(")

def isLastCharacterPunctuation(s):
    if(s[-1]=='!' or s[-1]=='?'):
        return 1
    return 0

def isLastTokenPositiveEmoticon(s):
    lastToken = s.split()[-1]
    if(getNumberOfPositiveSmileys(lastToken)>=1):
        return 1
    return 0

def isLastTokenNegativeEmoticon(s):
    lastToken = s.split()[-1]
    if(getNumberOfNegativeSmileys(lastToken)>=1):
        return 1
    return 0        

def getNumberOfPositiveWords(s):
    with open("positiveWords.txt", "r") as f:
        wordsFromS = set(s.split())
        wordsFromFile = set(x.rstrip() for x in f.readlines())
        #print wordsFromS
        return len(wordsFromFile.intersection(wordsFromS))

def getNumberOfNegativeWords(s):
    with open("negativeWords.txt", "r") as f:
        wordsFromS = set(s.split())
        wordsFromFile = set(x.rstrip() for x in f.readlines())
        #print wordsFromS
        return len(wordsFromFile.intersection(wordsFromS))

def READ_FILES():

        fp = codecs.open(PRONOUN_PATH,'r','utf-8')
        for l in fp:
                PRONOUN[l.strip(' \t\n\r')] = 1
        fp.close()

        fp = codecs.open(INTENSIFIER_PATH,'r','utf-8')
        for l in fp:
                INTENSIFIER[l.strip(' \t\n\r')] = 1
        fp.close()

        fp = codecs.open(WHWORD_PATH,'r','utf-8')
        for l in fp:
                WHWORD[l.strip(' \t\n\r')] = 1
        fp.close()

        fp = codecs.open(SLANG_PATH,'r','utf-8')
        for l in fp:
                SLANG[l.strip(' \t\n\r')] = 1
        fp.close()

        fp = codecs.open(EVENT_PATH,'r','utf-8')
        for l in fp:
                EVENT[l.strip(' \t\n\r')] = 1
        fp.close()

        fp = codecs.open(MODAL_VERB_PATH,'r','utf-8')
        for l in fp:
                MODAL[l.strip(' \t\n\r')] = 1
        fp.close()

        fp = codecs.open(RELIGION_PATH,'r','utf-8')
        for l in fp:
                RELIGION[l.strip(' \t\n\r')] = 1
        fp.close()

        fp = codecs.open(SUBJECTIVE_PATH,'r','utf-8')
        for l in fp:
		SUBJECTIVE[l.strip(' \t\n\r')] = 1
        fp.close()


#getNumberOfNegativeWords("aa b")    

############################ This Functions are used #############################################
def emoticons(s):
        return len(re.findall(u'[\U0001f600-\U0001f60f\U0001f617-\U0001f61d\U0001f632\U0001f633\U0001f638-\U0001f63e\U0001f642\U0001f646-\U0001f64f\U0001f612\U0001f613\U0001f615\U0001f616\U0001f61e-\U0001f629\U0001f62c\U0001f62d\U0001f630\U0001f631\U0001f636\U0001f637\U0001f63c\U0001f63f-\U0001f641\U0001f64d]', s))

def smileys(s):
        return len(re.findall(r':\-\)|:[\)\]\}]|:[dDpP]|:3|:c\)|:>|=\]|8\)|=\)|:\^\)|:\-D|[xX8]\-?D|=\-?D|=\-?3|B\^D|:\'\-?\)|>:\[|:\-?\(|:\-?c|:\-?<|:\-?\[|:\{|;\(|:\-\|\||:@|>:\(|:\'\-?\(|D:<?|D[8;=X]|v.v|D\-\':|>:[\/]|:\-[./]|:[\/LS]|=[\/L]|>.<|:\$|>:\-?\)|>;\)|\}:\-?\)|3:\-?\)|\(>_<\)>?|^_?^;|\(#\^.\^#\)|[Oo]_[Oo]|:\-?o',s))

def getNumberOfElongatedWords(s):
    return len(re.findall('([a-zA-Z])\\1{2,}', s))
    
def pronoun(sen):

	for x in sen:
                if PRONOUN.__contains__(x)==True:
                        return 1
        return 0

def exclamation(s):
	c = len(re.findall(r"[!]", s))
	if c>=1:
		return 1
	return 0

def question(s):
	return len(re.findall(r"[?]", s))

def intensifier(sen):
	count = 0
        for x in sen:
                if INTENSIFIER.__contains__(x)==True:
                        count+=1
                        #return 1
        if count>0:
                return 1
        return 0

def religion(sen):

        for x in sen:
                if RELIGION.__contains__(x)==True:
                        return 1
        return 0

def whword(sen):

	for x in sen:
                if WHWORD.__contains__(x)==True:
                        return 1
        return 0

def slang(sen):

	for x in sen:
                if SLANG.__contains__(x)==True:
                        return 1
        return 0

def event_phrase(sen):
	
	for x in sen:
                if EVENT.__contains__(x)==True:
                        return 1
        return 0

def getHashtagopinion(sen):
	fp = codecs.open(OPINION_HASHTAG_PATH,'r','utf-8')
	temp = set([])
	for l in fp:
		temp.add(l.strip(' \t\n\r').lower())
	fp.close()

	cur_hash = set([])
	for x in sen:
		if x.startswith('#')==True:
			cur_hash.add(x.strip(' \t\n\r').lower())
	size = len(temp.intersection(cur_hash))
	if size>0:
		return 1
	return 0

def numeral(temp):
	c = 0
	for x in temp:
		if x.isdigit()==True:
			c+=1
	return c

def getNewsMention(sen):
	fp = codecs.open(MENTION_PATH,'r','utf-8')
	temp = set([])
	for l in fp:
		temp.add(l.strip(' \t\n\r').lower())
	fp.close()

	cur_men = set([])
	for x in sen:
		if x.startswith('@')==True:
			cur_men.add(x.strip(' \t\n\r').lower())
	size = len(temp.intersection(cur_men))
	if size>0:
		return 1
	return 0
	
def modal(sen):

	for x in sen:
                if MODAL.__contains__(x)==True:
                        return 1
        return 0

def subjectivity(sen):
	c = 0
        for x in sen:
                if SUBJECTIVE.__contains__(x)==True:
                        c+=1
        tot = len(sen) + 4.0 - 4.0 - 1.0
        num = c + 4.0 - 4.0
        try:
                s = round(num/tot,4)
        except:
                s = 0
        if c>0:
                return c
        return 0

######################## Upto this part used #############################################################3

#HashTagSentimentUnigrams("Oh wrestle", "NRC-Hashtag-Sentiment-Lexicon-v0.1/unigrams-pmilexicon.txt")
#HashTagSentimentUnigrams("Oh no wrestle", "Sentiment140-Lexicon-v0.1/unigrams-pmilexicon.txt")
#print negatedContextCount("this is not for me")
            
if __name__ == '__main__':

	READ_FILES()
	'''org_tweet = {}
	fp = codecs.open(sys.argv[1],'r','utf-8')
	for l in fp:
		wl = l.split('\t')
		tid = wl[1].strip(' \t\n\r')
		s = wl[3].strip(' \t\n\r').split()
		text = ''
		for x in s:
			if len(re.findall(r'[a-zA-Z]+',x))>0:
				pass
			else:
				text = text + x + ' '
		#text = wl[3].strip(' \t\n\r')
		org_tweet[tid] = text.strip()
	fp.close()'''

	IN = []
	CR = []
	PR = []
	RC = []
	FS = []
	

	fs = codecs.open(sys.argv[2],'r','utf-8')
	fp = codecs.open(sys.argv[1],'r','utf-8')
	nepal_feature = []
	nepal_label = []
	count = 0
	temp = []
	N = 0
	#temp_raw = []
	for l in fs:
		wl = l.split('\t')
		if len(wl)>2:
			if wl[2].strip(' \t\n\r')!='SYM':
				temp.append(wl[0].strip(' \t\n\r'))
				if wl[2].strip(' \t\n\r')=='QC':
					N+=1
		else:
			if wl[0].strip(' \t\n\r')=='</s>':
				row = fp.readline().split('\t')
				bigram = []
				if len(temp)>1:
					for i in range(0,len(temp)-1,1):
						s = temp[i] + ' ' + temp[i+1]
						bigram.append(s)
				temp = temp + bigram
				#row = XL[3].strip(' \t\n\r')
				#temp = row[3].split()
				#N = numeral(temp)
				E = exclamation(row[3])
				Q = question(row[3])
				M = modal(temp)
				I = intensifier(temp)
				W = whword(temp)
				EP = event_phrase(temp)
				S = subjectivity(temp)
				SG = slang(temp)
				RL = religion(temp)
				P = pronoun(temp)
				#EM = emoticons(org_tweet[row[1].strip(' \t\n\r')])
				#SM = smileys(org_tweet[row[1].strip(' \t\n\r')])
				#t = [N,E,Q,M,I,W,S,P,EP,SG,EM,SM]
				t = [N,E,Q,M,I,W,S,P,EP,SG,RL]
				#t = [N,E,Q,M,I,W,S,P,SG,RL]
				nepal_feature.append(t)
				nepal_label.append(int(row[4]))
				count+=1
				'''if count==51:
					print(row[1].strip(' \t\n\r'))
					print(t)
					sys.exit(0)'''
				temp = []
				N = 0
			

	fp.close()
	fs.close()

	nepal_clf = svm.SVC()
	#nepal_clf = svm.SVC(kernel='rbf',gamma=0.5)
	#nepal_clf = svm.SVC(kernel='linear')
	#nepal_clf = svm.LinearSVC()
	#nepal_clf = LogisticRegression()
	#nepal_clf = BernoulliNB()
	#nepal_clf = RandomForestClassifier()
	nepal_clf.fit(nepal_feature,nepal_label)
	scores = cross_validation.cross_val_score(nepal_clf,nepal_feature,nepal_label,cv=10)
	print(scores.mean(),scores.std())
	#clf.cross_validation(feature,label)
	IN.append(scores.mean())
	

	fs = codecs.open(sys.argv[4],'r','utf-8')
	fp = codecs.open(sys.argv[3],'r','utf-8')
	harda_feature = []
	harda_label = []
	count = 0
	temp = []
	N = 0
	#temp_raw = []
	for l in fs:
		wl = l.split('\t')
		if len(wl)>2:
			if wl[2].strip(' \t\n\r')!='SYM':
				temp.append(wl[0].strip(' \t\n\r'))
				if wl[2].strip(' \t\n\r')=='QC':
					N+=1
		else:
			if wl[0].strip(' \t\n\r')=='</s>':
				row = fp.readline().split('\t')
				bigram = []
				if len(temp)>1:
					for i in range(0,len(temp)-1,1):
						s = temp[i] + ' ' + temp[i+1]
						bigram.append(s)
				temp = temp + bigram
				#row = XL[3].strip(' \t\n\r')
				#temp = row[3].split()
				#N = numeral(temp)
				E = exclamation(row[3])
				Q = question(row[3])
				M = modal(temp)
				I = intensifier(temp)
				W = whword(temp)
				EP = event_phrase(temp)
				S = subjectivity(temp)
				SG = slang(temp)
				RL = religion(temp)
				P = pronoun(temp)
				#EM = emoticons(org_tweet[row[1].strip(' \t\n\r')])
				#SM = smileys(org_tweet[row[1].strip(' \t\n\r')])
				#t = [N,E,Q,M,I,W,S,P,EP,SG,EM,SM]
				t = [N,E,Q,M,I,W,S,P,EP,SG,RL]
				#t = [N,E,Q,M,I,W,S,P,SG,RL]
				harda_feature.append(t)
				harda_label.append(int(row[4]))
				count+=1
				'''if count==51:
					print(row[1].strip(' \t\n\r'))
					print(t)
					sys.exit(0)'''
				temp = []
				N = 0

	fp.close()
	fs.close()

	harda_clf = svm.SVC()
	#harda_clf = svm.SVC(kernel='rbf',gamma=0.5)
	#harda_clf = svm.SVC(kernel='linear')
	#harda_clf = svm.LinearSVC()
	#harda_clf = LogisticRegression()
	#harda_clf = BernoulliNB()
	#harda_clf = RandomForestClassifier()
	harda_clf.fit(harda_feature,harda_label)
	scores = cross_validation.cross_val_score(harda_clf,harda_feature,harda_label,cv=10)
	print(scores.mean(),scores.std())
	#clf.cross_validation(feature,label)
	IN.append(scores.mean())

	print('Nepal - Harda: ', nepal_clf.score(harda_feature,harda_label))
	print('Harda - Nepal: ', harda_clf.score(nepal_feature,nepal_label))
	CR.append(nepal_clf.score(harda_feature,harda_label))
	CR.append(harda_clf.score(nepal_feature,nepal_label))
	
	harda_predicted_label = nepal_clf.predict(harda_feature)
        print('Nepal -> Harda')
        print('PRECISION: ',metrics.precision_score(harda_label,harda_predicted_label))
        print('RECALL: ',metrics.recall_score(harda_label,harda_predicted_label))
        print('F1_SCORE: ',metrics.f1_score(harda_label,harda_predicted_label))
	PR.append(metrics.precision_score(harda_label,harda_predicted_label))
	RC.append(metrics.recall_score(harda_label,harda_predicted_label))
	FS.append(metrics.f1_score(harda_label,harda_predicted_label))


	nepal_predicted_label = harda_clf.predict(nepal_feature)
        print('Harda -> Nepal')
        print('PRECISION: ',metrics.precision_score(nepal_label,nepal_predicted_label))
        print('RECALL: ',metrics.recall_score(nepal_label,nepal_predicted_label))
        print('F1_SCORE: ',metrics.f1_score(nepal_label,nepal_predicted_label))
	PR.append(metrics.precision_score(nepal_label,nepal_predicted_label))
	RC.append(metrics.recall_score(nepal_label,nepal_predicted_label))
	FS.append(metrics.f1_score(nepal_label,nepal_predicted_label))

	print(len(IN),len(CR),len(PR),len(RC),len(FS))
        print('Average Indomain Accuracy: ',np.mean(IN))
        print('Average Cross-domain Accuracy: ',np.mean(CR))
        print('Average and std Precision: ',np.mean(PR),np.std(PR))
        print('Average and std Recall: ',np.mean(RC),np.std(RC))
        print('Average and std F-score: ',np.mean(FS),np.std(FS))


In [5]:
import sys
import re
import codecs
fs = codecs.open('/home/sreenu/Desktop/rudra/koustav_phdthesis_2018-master/Chapter_3/hindi/classification/dataset/harda_devanagari_train.txt','r','utf-8')
fp = codecs.open('/home/sreenu/Desktop/rudra/koustav_phdthesis_2018-master/Chapter_3/hindi/classification/dataset/nepal_devanagari_train.txt','r','utf-8')
for i in fp:
    j=i.split('\t')
    print j[4]
fp.close()
fs.close()

1

2

2

2

2

2

1

1

2

1

2

1

2

1

1

1

2

1

2

2

1

1

1

1

1

2

2

1

2

1

2

2

1

1

2

1

1

1

2

1

2

2

1

1

2

1

1

1

1

2

1

1

2

1

1

2

2

2

1

1

2

1

1

2

1

1

2

1

1

1

1

1

1

2

2

1

2

2

1

2

1

2

2

2

2

1

2

2

1

1

2

2

2

1

1

1

1

2

1

1

2

1

2

1

2

1

2

1

1

1

2

1

2

1

2

2

1

2

2

2

1

2

2

1

1

2

2

1

2

1

1

1

1

2

1

1

1

2

2

1

2

1

1

1

1

2

1

1

1

2

1

1

2

2

1

1

1

2

2

1

2

1

1

1

1

1

1

1

1

2

2

2

1

2

1

2

2

2

2

2

1

2

2

1

2

2

2

1

2

2

2

1

2

1

2

2

2

1

2

2

1

1

2

1

2

2

2

2

1

2

1

2

1

1

2

2

2

2

1

2

1

1

2

1

1

1

1

2

2

1

2

2

2

1

2

2

1

2

1

1

1

2

2

2

2

2

1

1

1

1

2

1

1

1

2

2

1

2

2

2

2

1

2

2

1

1

1

2

2

1

2

2

1

1

1

2

2

1

1

2

2

2

1

1

1

2

2

2

2

1

2

2

1

2

2

2

2

1

1

2

2

1

2

1

2

1

1

2

1

2

1

2

1

1

1

1

2

2

2

2

2

1

2

1

1

1

1

2

1

2

1

1

1

1